In [19]:
import requests
from bs4 import BeautifulSoup, NavigableString
import math
import Levenshtein

In [82]:
def getQuotesByAuthor(author, page_num = None):
    old_author = author
    author = author.replace(" ", "+")
    all_quotes = []

    # if page number not specified, get true page number
    if page_num is None:
        try:
            page = requests.get("https://www.goodreads.com/quotes/search??utf8=%E2%9C%93&q=" + author + "&commit=Search")
            soup = BeautifulSoup(page.text, 'html.parser')
            pages = soup.find(class_="smallText").text
            of = pages.find("of ")
            showing = pages.find("Showing ")

            num_shown = pages[showing+10:of-1]
            total_num = pages[of+3:]
            total_num = total_num.replace(",", "").replace("\n", "")
            num_shown = int(num_shown)
            total_num = int(total_num)
            page_num = math.ceil(total_num/num_shown)
            print("looking through", page_num, "pages")
        except:
            page_num = 1
    
    # for each page
    for i in range(1, page_num+1, 1):

        try:
            page = requests.get("https://www.goodreads.com/quotes/search?commit=Search&page=" + str(i) + "&q=" + author + "&utf8=%E2%9C%93")
            soup = BeautifulSoup(page.text, 'html.parser')
            print("scraping page", i)
        except:
            print("could not connect to goodreads")
            break

        try:
            quote = soup.find(class_="leftContainer")
            quote_list = quote.find_all(class_="quoteDetails")
        except:
            pass

        # get data for each quote
        for quote in quote_list:

            meta_data = []
            # Get quote's author
            try:
                qAuthor = quote.find(class_="authorOrTitle").text
                qAuthor = qAuthor.replace(",", "")
                qAuthor = qAuthor.replace("\n", "")
                qAuthor = qAuthor.strip()
                qAuthor = qAuthor.replace(" ", "+")
                
                if(Levenshtein.distance(qAuthor,author) > 5):
                    continue

                else:
                    meta_data.append(author.strip())
            except:
                meta_data.append(None)
                continue

            # Get quote's text
            try:
                outer = quote.find(class_="quoteText")
                inner_text = [element for element in outer if isinstance(element, NavigableString)]
                inner_text = [x.replace("\n", "") for x in inner_text]
                final_quote = "\n".join(inner_text[:-4])
                meta_data.append(final_quote.strip())
            except:
                pass 

            
            # Get quote's book title
            try: 
                title = quote.find(class_="authorOrTitle")
                title = title.nextSibling.nextSibling.text
                # title = title.replace("\n", "")
                meta_data.append(title.strip())
                # print(title)
            except:
                meta_data.append(None)

            # Get quote's tags
            try:
                tags = quote.find(class_="greyText smallText left").text
                tags = [x.strip() for x in tags.split(',')]
                tags = tags[1:]
                meta_data.append(tags)
                # print(tags)
            except:
                meta_data.append(None)

            # Get number of likes
            try:
                likes = quote.find(class_="right").text
                likes = likes.replace("likes", "")
                likes = int(likes)
                meta_data.append(likes)
                # print(likes)
            except:
                meta_data.append(None)

            all_quotes.append(meta_data)


        for text, author, title, tags, likes in all_quotes:
            print(text)
            print(author)
            print(title)
            print(tags)
            print(likes)
            print()

    return all_quotes


In [83]:
all_quotes = []
author = "William Shakespeare"
old_author = author
page_num = 1;
author = author.replace(" ", "+")

page = requests.get("https://www.goodreads.com/quotes/search??utf8=%E2%9C%93&q=" + author + "&commit=Search")
soup = BeautifulSoup(page.text, 'html.parser')
pages = soup.find(class_="smallText").text
of = pages.find("of ")
showing = pages.find("Showing ")

num_shown = pages[showing+10:of-1]
total_num = pages[of+3:]
total_num = total_num.replace(",", "").replace("\n", "")
num_shown = int(num_shown)
total_num = int(total_num)
page_num = math.ceil(total_num/num_shown)
print("looking through", page_num, "pages")

looking through 308 pages


In [84]:
try:
    page = requests.get("https://www.goodreads.com/quotes/search?commit=Search&page=" + str(1) + "&q=" + author + "&utf8=%E2%9C%93")
    soup = BeautifulSoup(page.text, 'html.parser')
    print("scraping page", 1)
except:
    print("could not connect to goodreads")

try:
    quote = soup.find(class_="leftContainer")
    quote_list = quote.find_all(class_="quoteDetails")
except:
    pass

     # get data for each quote
for quote in quote_list:

    meta_data = []
            # Get quote's author
    try:
        qAuthor = quote.find(class_="authorOrTitle").text
        qAuthor = qAuthor.replace(",", "")
        qAuthor = qAuthor.replace("\n", "")
        qAuthor = qAuthor.strip()
        qAuthor = qAuthor.replace(" ", "+")

        if(Levenshtein.distance(qAuthor,author) > 5):
            continue
                
        else:
            meta_data.append(author.strip())
                # print(author)
    except:
        meta_data.append(None)

scraping page 1


In [85]:
try:
    outer = quote.find(class_="quoteText")
    inner_text = [element for element in outer if isinstance(element, NavigableString)]
    #inner_text = [x.replace("\n", "") for x in inner_text]
    final_quote = "\n".join(inner_text[:-4])
    meta_data.append(final_quote.strip())
except:
    pass 

            

In [78]:
qAuthor

'William+Shakespeare'

In [79]:
author

'William+Shakespeare'

In [80]:
Levenshtein.distance(author,qAuthor)

0

In [86]:
meta_data

['William+Shakespeare',
 '“These violent delights have violent ends\nAnd in their triump die, like fire and powder\nWhich, as they kiss, consume”']

In [65]:
getQuotesByAuthor("William Shakespeare",32)

scraping page 1
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
failed
pass
William+Shakespeare
“The fool doth think he is wise, but the wise man knows himself to be a fool.”  
  ―
As You Like It
[]
43206

William+Shakespeare
“Love all, trust a few, do wrong to none.”
All's Well That Ends Well
['love', 'trust', 'wrong']
35512

William+Shakespeare
“Love looks not with the eyes, but with the mind,
And therefore is winged Cupid painted blind.”
A Midsummer Night's Dream
[]
23218

William+Shakespeare
“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”
Twelfth Night
[]
15141

William+Shakespeare
“Doubt thou the stars are fire;
Doubt that the sun doth move;
Doubt truth to be a liar;
But never doubt I love.”
Hamlet
[]
9000

William+Shakespeare
“The fault, dear Brutus, is not in our stars, but in ourselves.”
Julius Caesar
['self-responsibility', 'timidity']
8676

William+Shakespeare
“Thi

scraping page 4
William+Shakespeare
“The fool doth think he is wise, but the wise man knows himself to be a fool.”  
  ―
As You Like It
[]
43206

William+Shakespeare
“Love all, trust a few, do wrong to none.”
All's Well That Ends Well
['love', 'trust', 'wrong']
35512

William+Shakespeare
“Love looks not with the eyes, but with the mind,
And therefore is winged Cupid painted blind.”
A Midsummer Night's Dream
[]
23218

William+Shakespeare
“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”
Twelfth Night
[]
15141

William+Shakespeare
“Doubt thou the stars are fire;
Doubt that the sun doth move;
Doubt truth to be a liar;
But never doubt I love.”
Hamlet
[]
9000

William+Shakespeare
“The fault, dear Brutus, is not in our stars, but in ourselves.”
Julius Caesar
['self-responsibility', 'timidity']
8676

William+Shakespeare
“This above all: to thine own self be true, 
And it must follow, as the night the day, 
Thou canst not the

scraping page 7
William+Shakespeare
“The fool doth think he is wise, but the wise man knows himself to be a fool.”  
  ―
As You Like It
[]
43206

William+Shakespeare
“Love all, trust a few, do wrong to none.”
All's Well That Ends Well
['love', 'trust', 'wrong']
35512

William+Shakespeare
“Love looks not with the eyes, but with the mind,
And therefore is winged Cupid painted blind.”
A Midsummer Night's Dream
[]
23218

William+Shakespeare
“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”
Twelfth Night
[]
15141

William+Shakespeare
“Doubt thou the stars are fire;
Doubt that the sun doth move;
Doubt truth to be a liar;
But never doubt I love.”
Hamlet
[]
9000

William+Shakespeare
“The fault, dear Brutus, is not in our stars, but in ourselves.”
Julius Caesar
['self-responsibility', 'timidity']
8676

William+Shakespeare
“This above all: to thine own self be true, 
And it must follow, as the night the day, 
Thou canst not the

scraping page 10
William+Shakespeare
“The fool doth think he is wise, but the wise man knows himself to be a fool.”  
  ―
As You Like It
[]
43206

William+Shakespeare
“Love all, trust a few, do wrong to none.”
All's Well That Ends Well
['love', 'trust', 'wrong']
35512

William+Shakespeare
“Love looks not with the eyes, but with the mind,
And therefore is winged Cupid painted blind.”
A Midsummer Night's Dream
[]
23218

William+Shakespeare
“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”
Twelfth Night
[]
15141

William+Shakespeare
“Doubt thou the stars are fire;
Doubt that the sun doth move;
Doubt truth to be a liar;
But never doubt I love.”
Hamlet
[]
9000

William+Shakespeare
“The fault, dear Brutus, is not in our stars, but in ourselves.”
Julius Caesar
['self-responsibility', 'timidity']
8676

William+Shakespeare
“This above all: to thine own self be true, 
And it must follow, as the night the day, 
Thou canst not th

scraping page 13
William+Shakespeare
“The fool doth think he is wise, but the wise man knows himself to be a fool.”  
  ―
As You Like It
[]
43206

William+Shakespeare
“Love all, trust a few, do wrong to none.”
All's Well That Ends Well
['love', 'trust', 'wrong']
35512

William+Shakespeare
“Love looks not with the eyes, but with the mind,
And therefore is winged Cupid painted blind.”
A Midsummer Night's Dream
[]
23218

William+Shakespeare
“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”
Twelfth Night
[]
15141

William+Shakespeare
“Doubt thou the stars are fire;
Doubt that the sun doth move;
Doubt truth to be a liar;
But never doubt I love.”
Hamlet
[]
9000

William+Shakespeare
“The fault, dear Brutus, is not in our stars, but in ourselves.”
Julius Caesar
['self-responsibility', 'timidity']
8676

William+Shakespeare
“This above all: to thine own self be true, 
And it must follow, as the night the day, 
Thou canst not th

...

scraping page 22
William+Shakespeare
“The fool doth think he is wise, but the wise man knows himself to be a fool.”  
  ―
As You Like It
[]
43206

William+Shakespeare
“Love all, trust a few, do wrong to none.”
All's Well That Ends Well
['love', 'trust', 'wrong']
35512

William+Shakespeare
“Love looks not with the eyes, but with the mind,
And therefore is winged Cupid painted blind.”
A Midsummer Night's Dream
[]
23218

William+Shakespeare
“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”
Twelfth Night
[]
15141

William+Shakespeare
“Doubt thou the stars are fire;
Doubt that the sun doth move;
Doubt truth to be a liar;
But never doubt I love.”
Hamlet
[]
9000

William+Shakespeare
“The fault, dear Brutus, is not in our stars, but in ourselves.”
Julius Caesar
['self-responsibility', 'timidity']
8676

William+Shakespeare
“This above all: to thine own self be true, 
And it must follow, as the night the day, 
Thou canst not th

scraping page 25
William+Shakespeare
“The fool doth think he is wise, but the wise man knows himself to be a fool.”  
  ―
As You Like It
[]
43206

William+Shakespeare
“Love all, trust a few, do wrong to none.”
All's Well That Ends Well
['love', 'trust', 'wrong']
35512

William+Shakespeare
“Love looks not with the eyes, but with the mind,
And therefore is winged Cupid painted blind.”
A Midsummer Night's Dream
[]
23218

William+Shakespeare
“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”
Twelfth Night
[]
15141

William+Shakespeare
“Doubt thou the stars are fire;
Doubt that the sun doth move;
Doubt truth to be a liar;
But never doubt I love.”
Hamlet
[]
9000

William+Shakespeare
“The fault, dear Brutus, is not in our stars, but in ourselves.”
Julius Caesar
['self-responsibility', 'timidity']
8676

William+Shakespeare
“This above all: to thine own self be true, 
And it must follow, as the night the day, 
Thou canst not th

scraping page 28
William+Shakespeare
“The fool doth think he is wise, but the wise man knows himself to be a fool.”  
  ―
As You Like It
[]
43206

William+Shakespeare
“Love all, trust a few, do wrong to none.”
All's Well That Ends Well
['love', 'trust', 'wrong']
35512

William+Shakespeare
“Love looks not with the eyes, but with the mind,
And therefore is winged Cupid painted blind.”
A Midsummer Night's Dream
[]
23218

William+Shakespeare
“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”
Twelfth Night
[]
15141

William+Shakespeare
“Doubt thou the stars are fire;
Doubt that the sun doth move;
Doubt truth to be a liar;
But never doubt I love.”
Hamlet
[]
9000

William+Shakespeare
“The fault, dear Brutus, is not in our stars, but in ourselves.”
Julius Caesar
['self-responsibility', 'timidity']
8676

William+Shakespeare
“This above all: to thine own self be true, 
And it must follow, as the night the day, 
Thou canst not th

scraping page 31
William+Shakespeare
“The fool doth think he is wise, but the wise man knows himself to be a fool.”  
  ―
As You Like It
[]
43206

William+Shakespeare
“Love all, trust a few, do wrong to none.”
All's Well That Ends Well
['love', 'trust', 'wrong']
35512

William+Shakespeare
“Love looks not with the eyes, but with the mind,
And therefore is winged Cupid painted blind.”
A Midsummer Night's Dream
[]
23218

William+Shakespeare
“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”
Twelfth Night
[]
15141

William+Shakespeare
“Doubt thou the stars are fire;
Doubt that the sun doth move;
Doubt truth to be a liar;
But never doubt I love.”
Hamlet
[]
9000

William+Shakespeare
“The fault, dear Brutus, is not in our stars, but in ourselves.”
Julius Caesar
['self-responsibility', 'timidity']
8676

William+Shakespeare
“This above all: to thine own self be true, 
And it must follow, as the night the day, 
Thou canst not th

[['William+Shakespeare',
  '“The fool doth think he is wise, but the wise man knows himself to be a fool.”  \n  ―',
  'As You Like It',
  [],
  43206],
 ['William+Shakespeare',
  '“Love all, trust a few, do wrong to none.”',
  "All's Well That Ends Well",
  ['love', 'trust', 'wrong'],
  35512],
 ['William+Shakespeare',
  '“Love looks not with the eyes, but with the mind,\nAnd therefore is winged Cupid painted blind.”',
  "A Midsummer Night's Dream",
  [],
  23218],
 ['William+Shakespeare',
  '“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”',
  'Twelfth Night',
  [],
  15141],
 ['William+Shakespeare',
  '“Doubt thou the stars are fire;\nDoubt that the sun doth move;\nDoubt truth to be a liar;\nBut never doubt I love.”',
  'Hamlet',
  [],
  9000],
 ['William+Shakespeare',
  '“The fault, dear Brutus, is not in our stars, but in ourselves.”',
  'Julius Caesar',
  ['self-responsibility', 'timidity'],
  8676],
 ['William+